# Privacy Audit - DPO Training (Stage 2)

Use DPO to perform preference optimization on the SFT model.

**Prerequisites:**
1. Enable Colab Uploading experimental feature in Kiro settings
2. Right-click to upload the following files to Colab:
   - `data/preference_data.jsonl`
   - `models/stage1_sft/` folder

## 1. Install Dependencies

In [1]:
!pip install -q datasets transformers peft trl accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 530.9/530.9 kB 31.8 MB/s eta 0:00:00


## 2. Check GPU

In [2]:
import torch
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
else:
    print("Warning: No GPU detected")

PyTorch version: 2.9.0+cu126
CUDA available: True
GPU: Tesla T4
GPU Memory: 15.8 GB


## 3. Check Uploaded Files

In [14]:
!ls /

bin			    home    mnt			      root	  tools
boot			    kaggle  NGC-DL-CONTAINER-LICENSE  run	  usr
content			    lib     opt			      sbin	  var
cuda-keyring_1.1-1_all.deb  lib32   preference_data.jsonl     srv
datalab			    lib64   proc		      stage1_sft
dev			    libx32  python-apt		      sys
etc			    media   python-apt.tar.xz	      tmp


In [13]:
import os

# 文件上传到 /content 目录
print("检查 / 目录:")
!ls -la /

print("\n检查 / 目录:")
!ls -la / 2>/dev/null || echo "data 目录不存在，请上传 data/preference_data.jsonl"

print("\n检 \stage1_sft 目录:")
!ls -la /stage1_sft/ 2>/dev/null || echo "models/stage1_sft 目录不存在，请上传该folder"

检查 / 目录:
total 2328
drwxr-xr-x   1 root root    4096 Feb  3 20:53 .
drwxr-xr-x   1 root root    4096 Feb  3 20:53 ..
lrwxrwxrwx   1 root root       7 Jun 27  2024 bin -> usr/bin
drwxr-xr-x   2 root root    4096 Apr 18  2022 boot
drwxr-xr-x   1 root root    4096 Feb  3 20:52 content
-rw-r--r--   1 root root    4332 Jul 10  2024 cuda-keyring_1.1-1_all.deb
drwxr-xr-x   1 root root    4096 Jan 30 14:22 datalab
drwxr-xr-x   6 root root     460 Feb  3 20:39 dev
-rwxr-xr-x   1 root root       0 Feb  3 20:39 .dockerenv
drwxr-xr-x   1 root root    4096 Feb  3 20:39 etc
drwxr-xr-x   2 root root    4096 Apr 18  2022 home
drwxr-xr-x   3 root root    4096 Feb  3 20:39 kaggle
lrwxrwxrwx   1 root root       7 Jun 27  2024 lib -> usr/lib
lrwxrwxrwx   1 root root       9 Jun 27  2024 lib32 -> usr/lib32
lrwxrwxrwx   1 root root       9 Jun 27  2024 lib64 -> usr/lib64
lrwxrwxrwx   1 root root      10 Jun 27  2024 libx32 -> usr/libx32
drwxr-xr-x   2 root root    4096 Jun 27  2024 media
drwxr-xr-x   2 root

<>:10: SyntaxWarning: invalid escape sequence '\s'
<>:10: SyntaxWarning: invalid escape sequence '\s'
/tmp/ipython-input-1702442188.py:10: SyntaxWarning: invalid escape sequence '\s'
  print("\n检 \stage1_sft 目录:")



检查 / 目录:
total 2328
drwxr-xr-x   1 root root    4096 Feb  3 20:53 .
drwxr-xr-x   1 root root    4096 Feb  3 20:53 ..
lrwxrwxrwx   1 root root       7 Jun 27  2024 bin -> usr/bin
drwxr-xr-x   2 root root    4096 Apr 18  2022 boot
drwxr-xr-x   1 root root    4096 Feb  3 20:52 content
-rw-r--r--   1 root root    4332 Jul 10  2024 cuda-keyring_1.1-1_all.deb
drwxr-xr-x   1 root root    4096 Jan 30 14:22 datalab
drwxr-xr-x   6 root root     460 Feb  3 20:39 dev
-rwxr-xr-x   1 root root       0 Feb  3 20:39 .dockerenv
drwxr-xr-x   1 root root    4096 Feb  3 20:39 etc
drwxr-xr-x   2 root root    4096 Apr 18  2022 home
drwxr-xr-x   3 root root    4096 Feb  3 20:39 kaggle
lrwxrwxrwx   1 root root       7 Jun 27  2024 lib -> usr/lib
lrwxrwxrwx   1 root root       9 Jun 27  2024 lib32 -> usr/lib32
lrwxrwxrwx   1 root root       9 Jun 27  2024 lib64 -> usr/lib64
lrwxrwxrwx   1 root root      10 Jun 27  2024 libx32 -> usr/libx32
drwxr-xr-x   2 root root    4096 Jun 27  2024 media
drwxr-xr-x   2 roo

## 4. 配置路径

In [15]:
import os
# 配置 - 使用上传到 /content 的文件
BASE_MODEL_NAME = "Qwen/Qwen2.5-0.5B-Instruct"
SFT_MODEL_DIR = "/stage1_sft"
PREFERENCE_DATA = "/preference_data.jsonl"
OUTPUT_DIR = "/content/models/stage2_dpo"

os.makedirs(OUTPUT_DIR, exist_ok=True)

print(f"Base model: {BASE_MODEL_NAME}")
print(f"SFT model: {SFT_MODEL_DIR}")
print(f"Preference data: {PREFERENCE_DATA}")
print(f"Output: {OUTPUT_DIR}")

# 验证文件存在
assert os.path.exists(PREFERENCE_DATA), f"Not found: {PREFERENCE_DATA}"
assert os.path.exists(f"{SFT_MODEL_DIR}/adapter_config.json"), "SFT adapter not found"
print("\n✅ All files verified!")

Base model: Qwen/Qwen2.5-0.5B-Instruct
SFT model: /stage1_sft
Preference data: /preference_data.jsonl
Output: /content/models/stage2_dpo

✅ All files verified!


## 5. 加载模型和数据

In [16]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel
from trl import DPOTrainer, DPOConfig

print("Loading tokenizer...")
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL_NAME)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

print("Loading base model...")
base_model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL_NAME,
    device_map="auto",
    torch_dtype=torch.bfloat16
)

print("Loading SFT adapter...")
model = PeftModel.from_pretrained(base_model, SFT_MODEL_DIR)
print("Model loaded!")

Loading tokenizer...
Loading base model...


Loading weights:   0%|          | 0/290 [00:00<?, ?it/s]

Loading SFT adapter...
Model loaded!


In [17]:
print("Loading preference dataset...")
dataset = load_dataset("json", data_files=PREFERENCE_DATA, split="train")
print(f"Dataset loaded: {len(dataset)} examples")

Loading preference dataset...


Generating train split: 0 examples [00:00, ? examples/s]

Dataset loaded: 1951 examples


## 6. 配置 DPO Trainer

In [20]:
# 禁用 gradient checkpointing 以避免 PEFT 兼容性问题
model.config.use_cache = False
if hasattr(model, 'gradient_checkpointing_disable'):
    model.gradient_checkpointing_disable()

dpo_config = DPOConfig(
    learning_rate=5e-5,
    num_train_epochs=1,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=8,
    output_dir=OUTPUT_DIR,
    logging_steps=10,
    save_steps=100,
    beta=0.1,
    max_length=512,
    max_prompt_length=256,
    bf16=True,
    gradient_checkpointing=False,
    remove_unused_columns=False,
)

trainer = DPOTrainer(
    model=model,
    ref_model=None,
    args=dpo_config,
    train_dataset=dataset,
    processing_class=tokenizer,
)
print("DPO Trainer ready!")


DPO Trainer ready!


## 7. 开始训练

In [21]:
print("Starting DPO training...")
trainer.train()

Starting DPO training...


Step,Training Loss
10,0.279012
20,0.149707
30,0.116781
40,0.058500
50,0.053687
60,0.056368
70,0.037057
80,0.035838
90,0.032211
100,0.046586


TrainOutput(global_step=122, training_loss=0.07806927909250141, metrics={'train_runtime': 1627.1313, 'train_samples_per_second': 1.199, 'train_steps_per_second': 0.075, 'total_flos': 0.0, 'train_loss': 0.07806927909250141, 'epoch': 1.0})

## 8. 保存模型

In [22]:
print("Saving model...")
trainer.save_model(OUTPUT_DIR)
tokenizer.save_pretrained(OUTPUT_DIR)
print(f"Model saved to {OUTPUT_DIR}")
print(os.listdir(OUTPUT_DIR))

Saving model...
Model saved to /content/models/stage2_dpo
['checkpoint-100', 'tokenizer_config.json', 'adapter_config.json', 'training_args.bin', 'checkpoint-122', 'README.md', 'tokenizer.json', 'chat_template.jinja', 'adapter_model.safetensors']


## 9. 打包下载

训练完成后，右键 `/content/models/stage2_dpo` folder下载到本地。

或者运行下面的 cell 打包成 zip：

In [23]:
import shutil
shutil.make_archive("/content/stage2_dpo", 'zip', OUTPUT_DIR)
print("Created /content/stage2_dpo.zip")
print("右键该文件下载到本地，解压到 models/stage2_dpo/")

Created /content/stage2_dpo.zip
右键该文件下载到本地，解压到 models/stage2_dpo/


In [24]:
print("🎉 DPO training complete!")

🎉 DPO training complete!
